In [67]:
import pandas as pd
import spacy
from spacy.matcher import Matcher
from spacy import displacy

In [4]:
nlp = spacy.load('en_core_web_lg')

In [2]:
data = pd.read_excel('l1_12_sentiment_modified.xlsx',sheet_name="FundPerformance")

In [3]:
data.head()

,Unnamed: 0,Verbatim,L1_Verbatim,L2,Sentiment
0,0,"i do recommend bsl frontline fund , bsl midcap...","i do recommend bsl frontline fund , bsl midcap...",Rate of returns,Positive
1,1,1 ) as an ifa the experience is very good . mo...,most of the funds like bsl frontline equity fu...,Consistency of returns,Positive
2,2,( 1 ) both equity and debt schemes of birla ar...,( 1 ) both equity and debt schemes of birla ar...,Consistency of returns,Positive
3,3,1 ) i am very happy with the services of bslmf...,but when i compare it with other amc for fund ...,Unspecified / Others,Netural / Negative
4,4,1 ) i am happy with the fund performance of bs...,1 ) i am happy with the fund performance of bs...,Unspecified / Others,Positive


In [6]:
doc = nlp(data.L1_Verbatim.values[0])

In [10]:
list(doc.noun_chunks)

[i,
 bsl frontline fund,
 bsl midcap fund,
 bsl dynamic bond fund,
 bsl,
 95 fund,
 good returns]

In [25]:
doc.to_json()

{'text': 'i do recommend bsl frontline fund , bsl midcap fund , bsl dynamic bond fund , bsl balanced 95 fund which are providing good returns .',
 'ents': [{'start': 91, 'end': 93, 'label': 'CARDINAL'}],
 'sents': [{'start': 0, 'end': 133}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 1,
   'pos': 'PRON',
   'tag': 'PRP',
   'dep': 'nsubj',
   'head': 2},
  {'id': 1,
   'start': 2,
   'end': 4,
   'pos': 'VERB',
   'tag': 'VBP',
   'dep': 'aux',
   'head': 2},
  {'id': 2,
   'start': 5,
   'end': 14,
   'pos': 'VERB',
   'tag': 'VB',
   'dep': 'ccomp',
   'head': 17},
  {'id': 3,
   'start': 15,
   'end': 18,
   'pos': 'ADJ',
   'tag': 'JJ',
   'dep': 'nmod',
   'head': 5},
  {'id': 4,
   'start': 19,
   'end': 28,
   'pos': 'ADJ',
   'tag': 'JJ',
   'dep': 'compound',
   'head': 5},
  {'id': 5,
   'start': 29,
   'end': 33,
   'pos': 'NOUN',
   'tag': 'NN',
   'dep': 'dobj',
   'head': 2},
  {'id': 6,
   'start': 34,
   'end': 35,
   'pos': 'PUNCT',
   'tag': ',',
   'dep': 'punct'

#https://spacy.io/usage/rule-based-matching

{'OP': '!'}	Negation: match 0 times

{'OP': '?'}	Optional: match 0 or 1 times

{'OP': '+'}	Match 1 or more times

{'OP': '*'}	Match 0 or more times

## Method 1

In [172]:
def get_all_matched_pharses(text, pattern):
    matcher = Matcher(nlp.vocab)

    matcher.add('IPHONE_PATTERN', None, pattern)

    doc = nlp(text)

    matches = matcher(doc)
    
    for match_id, start, end in matches: 
        span = doc[start:end]
        print(span.text)

In [173]:
pattern = [
    {'POS':'NOUN','OP':'+'}
]
#text = "2018 FIFA World Cup: France won!"

get_all_matched_pharses(data.L1_Verbatim.values[0],pattern)

fund
bsl
bsl midcap
midcap
bsl midcap fund
midcap fund
fund
bond
bond fund
fund
fund
returns


## Method 2

In [103]:
def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]  # Matched span
    sent = span.sent  # Sentence containing matched span
    # Append mock entity for match in displaCy style to matched_sents
    # get the match span by ofsetting the start and end of the span with the
    # start and end of the sentence in the doc
    match_ents = [{
        "start": span.start_char - sent.start_char,
        "end": span.end_char - sent.start_char,
        "label": "MATCH",
    }]
    matched_sents.append({"text": sent.text, "ents": match_ents})
    return matched_sents

def display_matched_pharses(text, pattern):
    matcher = Matcher(nlp.vocab)
    matcher.add("FacebookIs", collect_sents, pattern)  # add pattern
    doc = nlp(text)
    matches = matcher(doc)
    displacy.render(matched_sents, style="ent", manual=True)
    

In [104]:
matched_sents

[{'text': 'i do recommend bsl frontline fund , bsl midcap fund , bsl dynamic bond fund , bsl balanced 95 fund which are providing good returns .',
  'ents': [{'start': 0, 'end': 0, 'label': 'MATCH'}]},
 {'text': 'i do recommend bsl frontline fund , bsl midcap fund , bsl dynamic bond fund , bsl balanced 95 fund which are providing good returns .',
  'ents': [{'start': 2, 'end': 1, 'label': 'MATCH'}]},
 {'text': 'i do recommend bsl frontline fund , bsl midcap fund , bsl dynamic bond fund , bsl balanced 95 fund which are providing good returns .',
  'ents': [{'start': 5, 'end': 4, 'label': 'MATCH'}]},
 {'text': 'i do recommend bsl frontline fund , bsl midcap fund , bsl dynamic bond fund , bsl balanced 95 fund which are providing good returns .',
  'ents': [{'start': 15, 'end': 14, 'label': 'MATCH'}]},
 {'text': 'i do recommend bsl frontline fund , bsl midcap fund , bsl dynamic bond fund , bsl balanced 95 fund which are providing good returns .',
  'ents': [{'start': 15, 'end': 18, 'label'

In [114]:
matched_sents = []
#pattern = [{"LOWER": "facebook"}, {"LEMMA": "be"}, {"POS": "ADV", "OP": "*"},
#           {"POS": "ADJ"}]

#text = "I'd say that Facebook is evil. – Facebook is pretty cool, right?"
pattern = [
    {'TAG':'JJ','OP':'*'},
    {'TAG':'NN','OP':'+'},
]
    
display_matched_pharses(data.L1_Verbatim.values[0],pattern)

## Method 3

In [90]:
import re

In [124]:
def filtered_chunks_tag(doc, pattern):
  for chunk in doc.noun_chunks:
    signature = ''.join(['<%s>' % w.tag_ for w in chunk])
    if pattern.match(signature) is not None:
      yield chunk
    
def filtered_chunks_tag_all(texts, pattern):
    result = set()
    
    for text in texts:
        doc = nlp(text)
        t_r = list(filtered_chunks_tag(doc,pattern))
        

In [ ]:
def temp_fun(doc, pattern):
    signature = ''.join(['<%s>' % w.tag_ for w in doc])
    if pattern.match(signature) is not None:
      yield chunk

In [166]:
signature = ''.join(['<%s>' % w.tag_ for w in doc])
pattern = re.compile(r'(<JJ>)*(<NN>|<NNS>|<NNP>)+')
z = pattern.match(signature)

In [167]:
pattern.findall(signature)

[('<JJ>', '<NN>'), ('', '<NN>')]

In [131]:
def filtered_chunks_by_pos(doc, pattern):
  for chunk in doc.noun_chunks:
    signature = ''.join(['<%s>' % w.pos_ for w in chunk])
    if pattern.match(signature) is not None:
      yield chunk

In [142]:
data.L1_Verbatim.values[9]

'and i do recommend bsl frontline equity fund , bsl equity fund ,'

In [168]:
doc = nlp(data.L1_Verbatim.values[9])
pattern = re.compile(r'(<JJ>)*(<NN>|<NNS>|<NNP>)+')

print(list(filtered_chunks_tag(doc, pattern)))

[bsl frontline equity fund, bsl equity fund]


In [144]:
doc = nlp(data.L1_Verbatim.values[9])
pattern = re.compile(r'(<DET>)*(<NOUN>)+')

print(list(filtered_chunks_by_pos(doc, pattern)))

[bsl equity fund]


In [169]:
1+1

2